# Flux sampling

## Basic usage

The easiest way to get started with flux sampling is using the `sample` function in the `flux_analysis` submodule. `sample` takes at least two arguments: a cobra model and the number of samples you want to generate.

In [1]:
from cobra.test import create_test_model
from cobra.flux_analysis import sample

model = create_test_model("textbook")
s = sample(model, 100)
s.head()

/home/moritz/.virtualenvs/cobra/lib/python3.5/site-packages/optlang/gurobi_interface.py:26: UserWarning: Be careful! The GUROBI interface is still under construction ...
  warn("Be careful! The GUROBI interface is still under construction ...")


,ACALD,ACALDt,ACKr,ACONTa,ACONTb,ACt2r,ADK1,AKGDH,AKGt2r,ALCD2x,...,RPI,SUCCt2_2,SUCCt3,SUCDi,SUCOAS,TALA,THD2,TKT1,TKT2,TPI
0,-0.291556,-0.194709,-2.359507,9.759692,9.759692,-2.359507,12.066529,4.969962,-3.643735,-0.096847,...,-0.188677,7.998656,8.076692,717.081281,-4.969962,0.058297,1.711138,0.058297,0.005867,9.659238
1,-0.138311,-0.047694,-1.892554,9.672063,9.672063,-1.892554,33.199472,6.168586,-0.053427,-0.090617,...,-2.099542,1.506340,2.366091,622.923863,-6.168586,2.030610,38.034517,2.030610,2.002889,7.701617
2,-0.580313,-0.278247,-0.404753,11.999287,11.999287,-0.404753,38.805878,8.594194,-0.393892,-0.302066,...,-0.386299,3.375453,3.839394,671.972932,-8.594194,0.372344,42.368459,0.372344,0.366733,9.189918
3,-1.071017,-0.673946,-0.306338,6.295152,6.295152,-0.306338,18.968824,3.758716,-0.876761,-0.397071,...,-1.453765,1.831685,2.327740,533.970769,-3.758716,1.450092,13.625763,1.450092,1.448616,7.426540
4,-2.095375,-0.734578,-0.010589,12.770724,12.770724,-0.010589,9.633019,10.254673,-0.282577,-1.360797,...,-1.166260,54.370578,55.070421,616.680319,-10.254673,1.161119,6.521790,1.161119,1.159052,8.796084


By default sample uses the `optgp` method based on the [method presented here](http://dx.doi.org/10.1371/journal.pone.0086587) as it is suited for larger models and can run in parallel. By default the sampler uses a single process. This can be changed by using the `processes` argument.

In [2]:
print("One process:")
%time s = sample(model, 1000)
print("Two processes:")
%time s = sample(model, 1000, processes=2)

One process:
CPU times: user 5.37 s, sys: 568 ms, total: 5.94 s
Wall time: 5.3 s
Two processes:
CPU times: user 56 ms, sys: 192 ms, total: 248 ms
Wall time: 2.66 s


Alternatively you can also user Artificial Centering Hit-and-Run for sampling by setting the method to `arch`. `arch` does not support parallel execution but has good convergence and is almost Markovian.

In [3]:
s = sample(model, 100, method="arch")

In general setting up the sampler is expensive since initial search directions are generated by solving many linear programming problems. Thus, we recommend to generate as many samples as possible in one go. However, this might require finer control over the sampling procedure as described in the following section.

## Advanced usage

### Sampler objects

The sampling process can be controlled on a lower level by using the sampler classes directly.

In [4]:
from cobra.flux_analysis.sampling import OptGPSampler, ARCHSampler

Both sampler classes have standardized interfaces and take some additional argument. For instance the `thinning` factor. "Thinning" means only recording samples every n iterations. A higher thinning factors mean less correlated samples but also larger computation times. By default the samplers use a thinning factor of 100 which creates roughly uncorrelated samples. If you want less samples but better mixing feel free to increase this parameter. If you want to study convergence for your own model you might want to set it to 1 to obtain all iterates.

In [5]:
arch = ARCHSampler(model, thinning=10)

`OptGPSampler` has an additional `processes` argument specifying how many processes are used to create parallel sampling chains. This should be in the order of your CPU cores for maximum efficiency. As noted before class initialization can take up to a few minutes due to generation of initial search directions. Sampling on the other hand is quick.

In [6]:
optgp = OptGPSampler(model, processes=4)

### Sampling and validation

Both samplers have a sample function that generates samples from the initialized object and act like the `sample` function described above, only that this time it will only accept a single argument, the number of samples. For `OptGPSampler` the number of samples should be a multiple of the number of processes, otherwise it will be increased to the nearest multiple automatically.

In [7]:
s1 = arch.sample(100)

s2 = optgp.sample(100)

You can call `sample` repeatedly and both samplers are optimized to generate large amount of samples without falling into "numerical traps". All sampler objects have a `validate` function in order to check if a set of points are feasible and give detailed information about feasibility violations in a form of a short code denoting feasibility. Here the short code is a combination of any of the following letters:

- "v" - valid point
- "l" - lower bound violation
- "u" - upper bound violation
- "e" - equality violation (meaning the point is not a steady state)

For instance for a random flux distribution (should not be feasible):

In [8]:
import numpy as np

bad = np.random.uniform(-1000, 1000, size=len(model.reactions))
arch.validate(np.atleast_2d(bad))

array(['le'], 
      dtype='<U3')

And for our generated samples:

In [9]:
arch.validate(s1)

array(['v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v'], 
      dtype='<U3')

### Batch sampling

Sampler objects are made for generating billions of samples, however using the `sample` function might quickly fill up your RAM when working with genome-scale models. Here, the `batch` method of the sampler objects might come in handy. `batch` takes two arguments, the number of samples in each batch and the number of batches. This will make sense with a small example. 

Let's assume we want to quantify what proportion of our samples will grow. For that we might want to generate 10 batches of 50 samples each and measure what percentage of the individual 100 samples show a growth rate larger than 0.1. Finally, we want to calculate the mean and standard deviation of those individual percentages.

In [10]:
# Biomass reaction is reaction number 12
print(model.reactions[12])

Biomass_Ecoli_core


In [11]:
counts = [np.mean(s[:, 12] > 0.1) for s in optgp.batch(100, 10)]
print("Usually {:.2f}% +- {:.2f}% grow...".format(
    np.mean(counts) * 100.0, np.std(counts) * 100.0))

Usually 6.70% +- 0.78% grow...
